# packages

In [2]:
import time
import investpy
# https://investpy.readthedocs.io/_info/introduction.html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import pyfolio as pf #https://quantopian.github.io/pyfolio/ - need to install from git because of dependencies errors
from alpha_vantage.timeseries import TimeSeries
import yfinance as yf #https://pypi.org/project/yfinance/#description
import edhec_risk_kit_129 as erk
import requests
from io import StringIO
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

C:\Users\User\Anaconda3\envs\py37_trade\lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [12]:
df_admiral = pickle.load(open('data/df_admiral_yf.pickle', 'rb'))
df_admiral = df_admiral['Adj Close']
df_finax = pickle.load(open('data/df_finax.pickle', 'rb'))
df_finax = df_finax.tz_localize('UTC')
df_finax = df_finax['Adj Close']
df_admiral = df_admiral.tz_localize('UTC')

# correlations

In [13]:
df = df_finax.join(df_admiral, how = 'outer')
cm = sns.color_palette("coolwarm", as_cmap=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df.corr()[df_finax.columns].iloc[10:,:].style.background_gradient(cmap=cm, axis = None))

,DBZB.DE,FRCK.DE,IS3N.DE,SPY4.DE,SXR8.DE,XBLC.MI,XHYA.DE,XSX6.MI,XXSC.MI,ZPRR.DE
2INVE.MC,-0.791860,-0.669928,-0.894603,-0.797227,-0.778272,-0.836784,-0.803874,-0.894795,-0.872152,-0.865585
ACWI.PA,0.912920,0.728546,0.915571,0.982703,0.995142,0.957849,0.834353,0.962034,0.966382,0.931025
AEEM.PA,0.736315,0.765675,0.999456,0.897211,0.899051,0.810719,0.848618,0.871500,0.882992,0.913539
AGED.L,-0.795603,-0.189216,-0.353630,-0.234413,-0.617698,-0.509913,-0.176755,-0.162786,-0.179323,-0.249399
AUCO.AS,-0.331035,0.485893,0.581773,0.085151,-0.231954,-0.292611,0.447575,-0.423750,-0.386051,0.506944
AUT.PA,-0.313628,0.710967,0.807719,0.906475,0.541281,0.616415,0.853514,0.855868,0.862537,0.877940
BNK.PA,-0.161799,-0.077510,-0.199161,0.007830,-0.119571,-0.258299,-0.135044,0.077324,-0.081708,-0.223472
BNKE.PA,-0.811899,-0.043747,-0.224037,-0.449425,-0.635931,-0.596210,-0.124984,-0.253834,-0.316686,-0.249842
BRE.PA,0.262994,0.621920,0.935542,0.634452,0.385055,0.285092,0.782129,0.402094,0.319402,0.915472
BSX.PA,-0.926523,-0.751789,-0.902781,-0.940053,-0.918579,-0.964046,-0.875312,-0.971135,-0.955780,-0.899882


# similarities of pyfolio stats